# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [2]:
!pip install -q -U langchain openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.6 requires langchain<=0.0.266,>=0.0.262, but you have langchain 0.0.274 which is incompatible.


In [3]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key: ········


### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [4]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0.7)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [4]:
!pip install -q -U wikipedia

In [15]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs= 2,# YOUR CODE HERE, 
    doc_content_chars_max=4000### YOUR CODE HERE
    ).load()



ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=extracts&explaintext=&exintro=&titles=List+of+Barbie+animated+films&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7feed1313a00>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [16]:
barbie_csv_docs = CSVLoader(
    file_path='barbie_data/barbie.csv',### YOUR CODE HERE, 
    source_column='Review_Url'### YOUR CODE HERE
    ).load()

In [17]:
print(barbie_wikipedia_docs)

[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael Cera, Rhea Perlman, and Will Ferrell.\nA live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 2018. Robbie was cast in 2019, after Gal Gadot turned down the 

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000, ### YOUR CODE HERE,
    chunk_overlap =20, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators = '.'### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000, ### YOUR CODE HERE,
    chunk_overlap =20, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators ='\n' ### YOUR CODE HERE # keep paragraphs, then sentences
)


chunked_barbie_wikipedia_docs = wikipedia_text_splitter.create_documents([ doc.page_content for doc in barbie_wikipedia_docs])### YOUR CODE HERE
chunked_barbie_csv_docs = csv_text_splitter.create_documents([doc.page_content for doc in barbie_csv_docs]) ### YOUR CODE HERE

In [19]:
print(len(chunked_barbie_csv_docs))

161


In [20]:
print(chunked_barbie_csv_docs[0])

page_content=': 0\nReview_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy\nReview: Margot does the best with what she\'s given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I\'m a crotchety old man, I\'m a woman in my 20s, so I\'m pretty sure I\'m this movie\'s target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid\'s movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.\nReview_Url: /review/rw9199947/?ref_=tt_urv' metadata={}


#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [30]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [21]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./test_cache/") ### YOUR CODE HERE

core_embeddings_model = OpenAIEmbeddings() ### YOUR CODE HERE

embedder =  CacheBackedEmbeddings.from_bytes_store(core_embeddings_model, store, namespace=core_embeddings_model.model) ### YOUR CODE HERE

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [22]:
barbie_csv_faiss_retriever =  FAISS.from_documents(chunked_barbie_csv_docs, embedder) ### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [23]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
   chunked_barbie_wikipedia_docs ### YOUR CODE HERE
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
   chunked_barbie_wikipedia_docs, ### YOUR CODE HERE,
   embedder ### YOUR CODE HERE
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],### YOUR CODE HERE,
    weights= [0.5,0.5]### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [24]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
   barbie_wikipedia_faiss_retriever, ### YOUR CODE HERE 
   'search_barbie_on_wikipedia', ### YOUR CODE HERE
   'Searches and returns documents regarding the barbie on wikipedia.' ### YOUR CODE HERE
)

barbie_csv_retrieval_tool = create_retriever_tool(
   barbie_csv_faiss_retriever.as_retriever(), ### YOUR CODE HERE
   'search_barbie_on_csv', ### YOUR CODE HERE
   'Searches and returns documents regarding the barbie on provided csv' ### YOUR CODE HERE
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool,barbie_csv_retrieval_tool]### YOUR CODE HERE

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [25]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent


barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)# YOUR CODE HERE

In [26]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `search_barbie_on_wikipedia` with `{'query': 'Barbie'}`


[Document(page_content='.\n\n\n== Plot ==\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontinued models, who are treated like outcasts due to their unconventional traits. While the Kens spend their days playing at the beach, considering it as their profession, the Barbies hold prestigious jobs such as doctors, lawyers, and politicians. Beach Ken ("Ken") is only happy when he is with Barbie and seeks a closer relationship, but Barbie rebuffs him in favor of other activities and female friendships.\nOne evening at a dance party, Barbie is suddenly stricken with worries about mortality. Overnight, she develops bad breath, cellulite, and flat feet, disrupting her usual routines the next day. Weird Barbie, an outcast due to her disfigurement, tells her she must find the chil

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_barbie_on_wikipedia', 'arguments': '{\n  "query": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'.\\n\\n\\n== Plot ==\\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontinued models, who are treated like outcasts due to their unconventional traits. While the Kens spend their days playing at the beach, considering it as their profession, the Barbies hold prestigious jobs such as doctors, lawyers, and politicians. Beach Ken ("Ken

In [28]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...
The quick summary of the plot of the Barbie movie is that Barbie, a doll living in Barbieland, experiences an existential crisis that leads her to question her existence and the truth of Barbie Land.

> Finished chain.


{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_barbie_on_wikipedia', 'arguments': '{\n  "query": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'.\\n\\n\\n== Plot ==\\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontinued models, who are treated like outcasts due to their unconventional traits. While the Kens spend their days playing at the beach, considering it as their profession, the Barbies hold prestigious jobs such as doctors, lawyers, and politicians. Beach Ken ("Ken") is only happy when he is with Barbie and seeks a closer relat

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [38]:
oppenheimer_wikipedia_docs = WikipediaLoader(
query="Oppenheimer (film)",
load_max_docs=2,
doc_content_chars_max=4000
).load() ### YOUR CODE HERE

oppenheimer_csv_docs = CSVLoader(
file_path='oppenheimer_data/oppenheimer.csv',
source_column='Review_Url'
).load() ### YOUR CODE HERE

In [39]:
print(oppenheimer_wikipedia_docs)

[Document(page_content='Oppenheimer ( OP-ən-hy-mər) is a 2023 epic biographical thriller film written and directed by Christopher Nolan. Based on the 2005 biography American Prometheus by Kai Bird and Martin J. Sherwin, the film chronicles the career of American theoretical physicist J. Robert Oppenheimer. The story predominantly focuses on Oppenheimer\'s studies, his direction of the Manhattan Project during World War II, and his eventual fall from grace due to his 1954 security hearing. The film stars Cillian Murphy as the title character, Emily Blunt as his wife, "Kitty", Matt Damon as head of the Manhattan Project Leslie Groves, Robert Downey Jr. as U.S. Atomic Energy Commission member Lewis Strauss, and Florence Pugh as Communist Party USA member Jean Tatlock. The ensemble supporting cast includes Josh Hartnett, Casey Affleck, Rami Malek, and Kenneth Branagh.\nThe film was announced in September 2021 after Universal Pictures won a bidding war for Nolan\'s screenplay, following Nol

In [40]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.create_documents([ doc.page_content for doc in oppenheimer_wikipedia_docs])### YOUR CODE HERE
chunked_opp_csv_docs = csv_text_splitter.create_documents([doc.page_content for doc in oppenheimer_csv_docs]) ### YOUR CODE HERE

In [41]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder) ### YOUR CODE HERE

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(
chunked_opp_wikipedia_docs
) ### YOUR CODE HERE
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
chunked_opp_wikipedia_docs,
embedder
) ### YOUR CODE HERE
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],### YOUR CODE HERE,
    weights= [0.5,0.5]### YOUR CODE HERE # should sum to 1
)### YOUR CODE HERE

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [42]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [43]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever.as_retriever(),
    "question": lambda x: x["question"],
} | prompt | llm

In [44]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content='Based on the information from the sources, the public reviews about the Oppenheimer movie are mixed. Some viewers found the movie to be a great experience, compelling, engaging, and simple to understand. They praised the phenomenal performances of the actors, particularly Cillian Murphy, and the beautiful cinematography. These viewers also appreciated that the movie sparked an interest in history. One reviewer mentioned that they had read the book earlier and still enjoyed the film.\n\nHowever, there are also viewers who expressed disappointment with the movie. They felt that the film lacked emotion and failed to capture the complexity of J. Robert Oppenheimer as a person. They mentioned missing important relationships and moments of catharsis from the book. These viewers also felt that the film lacked heart and primarily focused on the Los Alamos project and testimonies.\n\nOverall, opinions on the Oppenheimer movie seem to be divided, with some viewers enjoying it 

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [60]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

def query_barbie(input):
    return barbie_retriever_agent_executor(input)

tools = [
    Tool(
        name ='search_oppenheimer_on_wikipedia', ### YOUR CODE HERE,
        func=query_oppenheimer,### YOUR CODE HERE,
        description='Searches and returns documents regarding the oppenheimer on wikipedia'### YOUR CODE HERE
    ),
    Tool(
        name ='search_barbie', ### YOUR CODE HERE,
        func=query_barbie, ### YOUR CODE HERE,
        description='Searches and returns documents regarding barbie' ### YOUR CODE HERE
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [61]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    prefix=prefix,### YOUR CODE HERE
    suffix=suffix,### YOUR CODE HERE
    input_variables=['input','agent_scratchpad'],### YOUR CODE HERE
    tools=tools### YOUR CODE HERE
)

In [62]:
from langchain import LLMChain

llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt)### YOUR CODE HERE

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [63]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain,### YOUR AGENTS, 
    tools=tools, ### YOUR AGENTS, 
    verbose=True ### YOUR AGENTS
)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent,### YOUR AGENTS, 
    tools=tools,### YOUR AGENTS, 
    verbose=True ### YOUR AGENTS
)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [64]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have the information about the Barbie movie.
Action: search_barbie
Action Input: None

> Entering new AgentExecutor chain...
Alright, if you have any questions in the future, feel free to ask. Have a great day!

> Finished chain.

Observation: {'input': 'None', 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_barbie_on_wikipedia', 'arguments': '{\n  "query": "Barbie"\n}'}}, example=False), FunctionMessage(content='[Document(page_content=\'.\\n\\n\\n== Plot ==\\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontinued models, who are treated like outcasts due to their unconventional traits. While t

{'input': 'What did people like about the Barbie movie?',
 'output': 'People have different opinions about the Barbie movie. Some find it to be more than just a toy and consider it a significant and famous topic. Others mention that Barbie has been dragged into cultural wars and used for political militancy. The plot of the Barbie movie revolves around Barbie experiencing an existential crisis that makes her question her existence and the truth of Barbie Land.'}

In [66]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Oppenheimer movie.
Action: search_oppenheimer_on_wikipedia
Action Input: Oppenheimer movie
Observation: content='The Oppenheimer movie is a 2023 biographical thriller film written and directed by Christopher Nolan. It is based on the 2005 biography "American Prometheus" by Kai Bird and Martin J. Sherwin. The film depicts the career of American theoretical physicist J. Robert Oppenheimer, focusing on his studies, his direction of the Manhattan Project during World War II, and his fall from grace due to his 1954 security hearing.\n\nThe movie stars Cillian Murphy as J. Robert Oppenheimer, Emily Blunt as his wife Kitty, Matt Damon as Leslie Groves (head of the Manhattan Project), Robert Downey Jr. as Lewis Strauss (U.S. Atomic Energy Commission member), and Florence Pugh as Jean Tatlock (Communist Party USA member). The ensemble supporting cast includes Josh Hartnett, Casey Affleck, Rami Malek, and Ken

'People liked the Oppenheimer movie for its storyline, direction, performances, cinematography, and exploration of ethical and moral dilemmas. It has been described as awe-inspiring, intellectually stimulating, and a masterpiece by reviewers.'

In [70]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the movies Barbie and Oppenheimer to determine if they share similar themes or ideas.
Action: search_barbie
Action Input: None

> Entering new AgentExecutor chain...
Alright, if you have any questions or need assistance in the future, feel free to ask. Have a great day!

> Finished chain.

Observation: {'input': 'None', 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_barbie_on_wikipedia', 'arguments': '{\n  "query": "Barbie"\n}'}}, example=False), FunctionMessage(content='[Document(page_content=\'.\\n\\n\\n== Plot ==\\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontin

Action Input: None
Observation: content="I'm sorry, but I cannot answer any questions with the given information. The first source provided does not contain any specific information or details, and the second source seems to be incomplete as it only includes review URLs. If you have any other information or specific questions, please provide them and I'll do my best to assist you." additional_kwargs={} example=False
Thought:I couldn't find any specific information about the movie Oppenheimer. I need to find another source or ask for more specific information to determine if it shares similar themes or ideas with the movie Barbie.
Action: search_oppenheimer_on_wikipedia
Action Input: None
Observation: content="I'm sorry, but it seems that there are no specific questions to answer based on the provided information from the two sources. If you have any questions or need assistance with a specific topic, please let me know and I'll be happy to help." additional_kwargs={} example=False
Thou

"I couldn't find any information about whether the movies Barbie and Oppenheimer share similar themes or ideas."

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: